In [57]:
from keras.models import load_model
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error,mean_squared_error
import json

In [58]:
model_dir="D:\\programs\\ads_flask\\logres_op\\model\\"
data_dir="D:\\programs\\ads_flask\\logres_op\\data\\"

In [59]:
ls_d=[]
files=[]
for root, dirs, files in os.walk(data_dir, topdown=False):
   for name in files:
      ls_d.append(os.path.join(root, name))
for i in range(len(files)):
   files[i]=files[i].replace(".csv","")

In [60]:
ls_m=[]
files=[]
for root, dirs, files in os.walk(model_dir, topdown=False):
   for name in files:
      ls_m.append(os.path.join(root, name))
for i in range(len(files)):
   files[i]=files[i].replace(".h5","")

In [61]:
def mae(yac90,ypr90):
    maesum=0
    for i in range(yac90.shape[1]):
        maesum+=mean_absolute_error(yac90[i],ypr90[i])
    maesum/=yac90.shape[1]
    return maesum
def rmse(yac90,ypr90):
    rmsesum=0
    for i in range(yac90.shape[1]):
        rmsesum+=mean_squared_error(yac90[i],ypr90[i],squared=False)
    rmsesum/=yac90.shape[1]
    return rmsesum
time=90

In [66]:
files[36]

'POWERGRID'

In [67]:
df=pd.read_csv(ls_d[36])

In [74]:
plt.figure(figsize=(25,10))
pd.DataFrame(df["close "]-df["close "].shift(1)).quantile(0.1)

close    -3.05
Name: 0.1, dtype: float64

<Figure size 2500x1000 with 0 Axes>

In [44]:
op=[]
for h in range(len(ls_m)):    
    df=pd.read_csv(ls_d[h])
    m=load_model(ls_m[h])


    data=[]
    for i in range(90,df["OPEN "].size):
        data.append(np.array(df["logret"][i-90:i]))
    data=pd.DataFrame(data)


    x_t=[]
    y_t=[]
    for i in range(data.shape[0]-1):
        x_t.append(data.T[i])
        y_t.append(data.T[i+1])
    x_t=np.array(x_t).reshape(data[0].size-1,time,1)
    y_t=np.array(y_t).reshape(data[0].size-1,time,1)


    ypr=m.predict(x_t)

    prte=[]
    for i in range(0,ypr.shape[0],time):
        prte.append(ypr[i])
    prte=np.array(prte,dtype='object')
    acte=[]
    for i in range(0,y_t.shape[0],time):
        acte.append(y_t[i])
    acte=np.array(acte,dtype='object')

    valac=[]
    for i in range(0,acte.shape[0]):
        t=[]
        for j in range(acte[i].shape[0]):
            if(j==0):
                t.append(df["close "][i*time]*(math.e**acte[i][j]))
            else:
                t.append(math.e**acte[i][j]*t[j-1])
        valac.append(t)
    valac=pd.DataFrame(valac)
    valac
    valpr=[]
    for i in range(0,prte.shape[0]):
        t=[]
        for j in range(prte[i].shape[0]):
            if(j==0):
                t.append(df["close "][i*time]*(math.e**prte[i][j]))
            else:
                t.append(math.e**prte[i][j]*t[j-1])
        valpr.append(t)
    valpr=pd.DataFrame(valpr)
    valpr

    op.append([files[h],mae(valac.T,valpr.T),rmse(valac.T,valpr.T)])

75/75 [==============================] - 6s 56ms/step


In [48]:
pd.DataFrame(op,columns=["stock","MAE","RMSE"]).to_csv("metrics.csv")

In [50]:
x=pd.read_csv("./metrics.csv")

In [52]:
x.shape

(50, 4)

In [53]:
y={}
for i in range(x.shape[0]):
    y[x["stock"][i]]=(x["MAE"][i],x["RMSE"][i])

In [56]:
with open('metrics.txt','w') as fp:
    fp.write(json.dumps(y))